In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [2]:
source = 'source/kilm11.xlsx'

In [3]:
data = pd.read_excel(source, skiprows=2)

In [4]:
data.head()

,Country (code),Country,Region,Sub-region (broad),Sub-region (detailed),Income group (code),Income group,B,Year,Sex (code),...,Repository,Type of source (code),Type of source,Coverage (code),Coverage,Reference period,Geographic limitation,Coverage limitation,Survey limitation,Remark
0,ALB,Albania,Europe and Central Asia,"Northern, Southern and Western Europe",Southern Europe,III,Upper-middle income,NaN,2009,MF,...,"ILOSTAT, online database",LFS,Labour force survey,C,Civilian,Noncalendar year,NaN,Excluding institutional population,NaN,NaN
1,ALB,Albania,Europe and Central Asia,"Northern, Southern and Western Europe",Southern Europe,III,Upper-middle income,NaN,2009,MF,...,"ILOSTAT, online database",LFS,Labour force survey,C,Civilian,Noncalendar year,NaN,Excluding institutional population,NaN,NaN
2,ALB,Albania,Europe and Central Asia,"Northern, Southern and Western Europe",Southern Europe,III,Upper-middle income,NaN,2009,MF,...,"ILOSTAT, online database",LFS,Labour force survey,C,Civilian,Noncalendar year,NaN,Excluding institutional population,NaN,NaN
3,ALB,Albania,Europe and Central Asia,"Northern, Southern and Western Europe",Southern Europe,III,Upper-middle income,NaN,2009,M,...,"ILOSTAT, online database",LFS,Labour force survey,C,Civilian,Noncalendar year,NaN,Excluding institutional population,NaN,NaN
4,ALB,Albania,Europe and Central Asia,"Northern, Southern and Western Europe",Southern Europe,III,Upper-middle income,NaN,2009,M,...,"ILOSTAT, online database",LFS,Labour force survey,C,Civilian,Noncalendar year,NaN,Excluding institutional population,NaN,NaN


In [24]:
data.columns

Index(['Country (code)', 'Country', 'Region', 'Sub-region (broad)',
       'Sub-region (detailed)', 'Income group (code)', 'Income group', 'B',
       'Year', 'Sex (code)', 'Sex', 'Age group (code)', 'Age group', 'Age',
       'Long-term unemployed ('000)', 'Total unemployed ('000)',
       'Labour force ('000)', 'Long-term unemployment rate (%)',
       'Incidence of long-term unemployment (%)', 'Repository (code)',
       'Repository', 'Type of source (code)', 'Type of source',
       'Coverage (code)', 'Coverage', 'Reference period',
       'Geographic limitation', 'Coverage limitation', 'Survey limitation',
       'Remark'],
      dtype='object')

In [5]:
# country

In [6]:
country = data[['Country (code)', 'Country']].drop_duplicates().copy()

In [7]:
country.columns = ['country', 'name']

In [8]:
country['country'] = country['country'].map(to_concept_id)

In [9]:
country.to_csv('../ddf--entities--country.csv', index=False)

In [11]:
# age group

In [17]:
age = data[['Age group (code)', 'Age group']].drop_duplicates().copy()
age.columns = ['age_group', 'name']
age['name'] = 'Age ' + age['name']
age['age_group'] = age['age_group'].map(to_concept_id)

In [18]:
age

,age_group,name
0,total,Age 15+
1,youth,Age 15-24
2,adult,Age 25+


In [19]:
age.to_csv('../ddf--entities--age_group.csv', index=False)

In [20]:
# sex

In [21]:
sex = data[['Sex (code)', 'Sex']].drop_duplicates().copy()
sex.columns = ['sex', 'name']

sex['sex'] = sex['sex'].map(to_concept_id)

sex.to_csv('../ddf--entities--sex.csv', index=False)


In [22]:
sex

,sex,name
0,mf,Male and female
3,m,Male
6,f,Female


In [23]:
# concepts

In [27]:
discs = ['Name', 'Year', 'Country', 'Sex', 'Age Group']

conc = data.columns[14:19]

In [26]:
conc

Index(['Long-term unemployed ('000)', 'Total unemployed ('000)',
       'Labour force ('000)', 'Long-term unemployment rate (%)',
       'Incidence of long-term unemployment (%)'],
      dtype='object')

In [28]:
cdf = pd.DataFrame([], columns=['concept', 'name', 'concept_type'])

cdf['name'] = [*discs, *conc]

cdf['concept'] = cdf['name'].map(to_concept_id)

In [29]:
cdf.loc[5:, 'concept_type'] = 'measure'
cdf.loc[0, 'concept_type'] = 'string'
cdf.loc[1, 'concept_type'] = 'time'
cdf.loc[2:4, 'concept_type'] = 'entity_domain'


In [30]:
cdf

,concept,name,concept_type
0,name,Name,string
1,year,Year,time
2,country,Country,entity_domain
3,sex,Sex,entity_domain
4,age_group,Age Group,entity_domain
5,long_term_unemployed_000,Long-term unemployed ('000),measure
6,total_unemployed_000,Total unemployed ('000),measure
7,labour_force_000,Labour force ('000),measure
8,long_term_unemployment_rate,Long-term unemployment rate (%),measure
9,incidence_of_long_term_unemployment,Incidence of long-term unemployment (%),measure


In [31]:
cdf.to_csv('../ddf--concepts.csv', index=False)

In [ ]:
# datapoints

In [35]:
dps = data[['Country (code)', 'Sex (code)', 'Age group (code)','Year', *conc]].copy()

In [36]:
dps.columns = ['country', 'sex', 'age_group', 'year', *[to_concept_id(x) for x in conc]]

In [37]:
dps['country'] = dps['country'].map(to_concept_id)
dps['sex'] = dps['sex'].map(to_concept_id)
dps['age_group'] = dps['age_group'].map(to_concept_id)

In [40]:
dps = dps.set_index(['country', 'sex', 'age_group', 'year'])

In [43]:

for k, df in dps.items():
    df_ = df.reset_index().dropna()
    path = '../ddf--datapoints--{}--by--country--sex--age_group--year.csv'.format(k)
    df_.to_csv(path, index=False)

In [42]:
create_index_file('../')

,key,value,file
0,concept,name,ddf--concepts.csv
1,concept,concept_type,ddf--concepts.csv
0,"country,sex,age_group,year",incidence_of_long_term_unemployment,ddf--datapoints--incidence_of_long_term_unempl...
0,"country,sex,age_group,year",labour_force_000,ddf--datapoints--labour_force_000--by--country...
0,"country,sex,age_group,year",long_term_unemployed_000,ddf--datapoints--long_term_unemployed_000--by-...
0,"country,sex,age_group,year",long_term_unemployment_rate,ddf--datapoints--long_term_unemployment_rate--...
0,"country,sex,age_group,year",total_unemployed_000,ddf--datapoints--total_unemployed_000--by--cou...
0,age_group,name,ddf--entities--age_group.csv
0,country,name,ddf--entities--country.csv
0,sex,name,ddf--entities--sex.csv
